# 多样本显著性检验——方差分析以及非参数检验

In [1]:
import pandas as pd
import pingouin as pg

df1 = pd.read_excel("data\\anova2.xlsx")
df1.head()

,Blend,Crop,Yield
0,Blend X,Wheat,123
1,Blend X,Wheat,156
2,Blend X,Wheat,112
3,Blend X,Wheat,100
4,Blend X,Corn,128


In [3]:
df2 = pd.read_excel("data\\rm_anova2.xlsx")
df2.head()

,Subject,Time,Metric,Performance
0,1,Pre,Product,13
1,2,Pre,Product,12
2,3,Pre,Product,17
3,4,Pre,Product,12
4,5,Pre,Product,19


### 1. 单因素方差分析（One-Way ANOVA）

**目的和用处：**
- **目的：** 比较一个因素（或变量）在三个或三个以上不同水平（组别）之间的平均数是否有显著差异。
- **用处：** 用于确定一个分类因素对观测变量的影响是否显著，例如不同教育水平对收入的影响、不同治疗方法对健康状况的影响等。

**模型假设：**
- **独立性假设：** 不同组别之间的观测值是相互独立的。
- **正态性假设：** 每个组别内的观测值来自于正态分布。
- **方差齐性假设：** 不同组别内的观测值具有相等的方差。


In [4]:
df1.head()

,Blend,Crop,Yield
0,Blend X,Wheat,123
1,Blend X,Wheat,156
2,Blend X,Wheat,112
3,Blend X,Wheat,100
4,Blend X,Corn,128


In [5]:
# 各组正态性检验
print(df1.groupby('Crop')['Yield'].apply(pg.normality).reset_index().drop('level_1', axis=1))

# 组间方差齐性检验
print(pg.homoscedasticity(df1, dv='Yield', group='Crop'))

    Crop         W      pval  normal
0   Corn  0.893707  0.253261    True
1    Soy  0.977271  0.948163    True
2  Wheat  0.949048  0.701627    True
               W      pval  equal_var
levene  0.856989  0.438757       True


In [6]:
# 如果数据满足ANOVA的假设（方差齐性和正态性），则进行单因素方差分析
pg.anova(dv='Yield', between='Crop', data=df1)

,Source,ddof1,ddof2,F,p-unc,np2
0,Crop,2,21,2.371707,0.117844,0.184257



### 2. 二因素方差分析（Two-Way ANOVA）

**目的和用处：**
- **目的：** 比较两个因素（或变量）对观测变量的联合影响是否显著，包括各自主效应以及交互效应（两个因素联合起来对结果的影响是否显著）。
- **用处：** 用于确定两个或多个因素之间的相互作用效应，例如性别和教育水平对收入的联合影响等。

**模型假设：**
- **独立性假设：** 不同组别之间的观测值是相互独立的。
- **正态性假设：** 每个组别内的观测值来自于正态分布。
- **方差齐性假设：** 不同组别内的观测值具有相等的方差。


In [7]:
# 各组正态性检验
print(df1.groupby('Crop')['Yield'].apply(pg.normality).reset_index().drop('level_1', axis=1))
print(df1.groupby('Blend')['Yield'].apply(pg.normality).reset_index().drop('level_1', axis=1))

# 组间方差齐性检验
print(pg.homoscedasticity(df1, dv='Yield', group='Crop'))
print(pg.homoscedasticity(df1, dv='Yield', group='Blend'))

    Crop         W      pval  normal
0   Corn  0.893707  0.253261    True
1    Soy  0.977271  0.948163    True
2  Wheat  0.949048  0.701627    True
     Blend         W      pval  normal
0  Blend X  0.947000  0.593617    True
1  Blend Y  0.888746  0.113534    True
               W      pval  equal_var
levene  0.856989  0.438757       True
               W      pval  equal_var
levene  0.830068  0.372128       True


In [8]:
pg.anova(dv='Yield', between=['Blend', 'Crop'], data=df1)

,Source,SS,DF,MS,F,p-unc,np2
0,Blend,2.041667,1,2.041667,0.003768,0.951730,0.000209
1,Crop,2736.583333,2,1368.291667,2.525235,0.107978,0.219105
2,Blend * Crop,2360.083333,2,1180.041667,2.177813,0.142223,0.194834
3,Residual,9753.250000,18,541.847222,NaN,NaN,NaN



### 3. 重复测量方差分析（Repeated Measures ANOVA）

**目的和用处：**
- **目的：** 比较同一组个体在不同时间点、不同处理条件或不同测量条件下的观测变量是否有显著变化。
- **用处：** 用于分析重复测量设计下的变化趋势或处理效应，例如在治疗前后观察患者的生理指标变化。

**模型假设：**
- **相关性假设：** 同一组个体在不同时间点或条件下的观测值是相关的。
- **正态性假设：** 每个时间点或条件下的观测值来自于正态分布。
- **同质性假设：** 不同时间点或条件下的观测值具有相等的协方差。


In [9]:
# 各组正态性检验
print(df2.groupby('Time')['Performance'].apply(pg.normality).reset_index().drop('level_1', axis=1))

# 进行Mauchly's Test来检验同质性假设
print(pg.sphericity(data=df2, dv='Performance', subject='Metric', within='Time'))

   Time         W      pval  normal
0  Post  0.940728  0.095158    True
1   Pre  0.967718  0.478771    True
(True, nan, nan, 1, 1.0)


In [10]:
pg.rm_anova(dv='Performance', 
        within='Time', 
        subject='Metric', 
        data=df2,
        detailed=True
        )

,Source,SS,DF,MS,F,p-unc,ng2,eps
0,Time,82.881667,1,82.881667,7.385861,0.112919,0.342701,1.0
1,Error,22.443333,2,11.221667,NaN,NaN,NaN,NaN



### 4. Kruskal-Wallis 检验（方差分析非参数检验）

- **目的和用处：**
  - **目的：** 比较多个独立组别（或条件）的中位数是否有显著差异。
  - **用处：** 适用于数据不满足正态分布或方差齐性假设的情况。

- **模型假设：**
  - **独立性假设：** 不同组别的样本相互独立。
  - **同质性假设：** 不同组别的样本来自相同的总体分布。


In [11]:
# Kruskal-Wallis检验
pg.kruskal(dv='Yield', between='Crop', data=df1)

,Source,ddof1,H,p-unc
Kruskal,Crop,2,4.50767,0.104996



### 5. Friedman 检验（重复测量方差分析非参数检验）

- **目的和用处：**
  - **目的：** 比较同一组体系中的多个条件或时间点下的中位数是否存在显著差异。
  - **用处：** 适用于重复测量设计数据不满足正态分布假设的情况。

- **模型假设：**
  - **相关性假设：** 同一组体系中的观测值是相关的。
  - **同质性假设：** 不同条件或时间点下的观测值来自同一总体分布。



In [12]:
pg.friedman(dv='Performance', 
        within='Time', 
        subject='Metric', 
        data=df2,
        )

,Source,W,ddof1,Q,p-unc
Friedman,Time,1.0,1,3.0,0.083265
